In [1]:
HF_USR_NAME = 'shirwu'
TOOL_QA_ROOT = ''

### Upload to Huggingface

In [2]:
import pandas as pd
from datasets import Dataset
from datasets import DatasetDict

level = 'hard'
dataset = 'agenda'

dataset_dir = f'{dataset}-{level}.jsonl'
hf_dataset_name = f'toolqa_{dataset}_{level}'

df = pd.read_json(dataset_dir, lines=True)
df.head()

df['answer'] = df['answer'].apply(lambda x: str(x))
dataset = Dataset.from_pandas(df)

In [3]:
dataset_dict = DatasetDict({'train': dataset})
# push to hf for the ease for using dspy
# dataset_dict.push_to_hub(repo_id=hf_dataset_name, private=True)

## Setting Up

* ToolQA

Before loading our datasets and going to the execution part, we'll need to configure the `lm` in `dspy.settings`. For the purpose of this notebook we'll be using `gpt-4o`.

In [4]:
import os
import dspy
import warnings
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", FutureWarning)


dspy.settings.configure(
    lm=dspy.OpenAI(
        model="gpt-4o",
        api_key=os.getenv("OPENAI_API_KEY"),
        max_tokens=4000,
        temperature=0
    )
)

## Defining Signature

In [5]:
class ToolQASignature(dspy.Signature):
    """You will be given a question. Your task is to answer the question with a short response. For example, (1) Question: \'How many dates in the agenda table have Alexander scheduled?\' => Answer: \'73\' (2) Question: \'What events does Jade have on 2022/01/25 in the agenda table?\' => Answer: \'Kids concert\'
    """
    
    question: str = dspy.InputField(
        prefix="Question:",
        desc="question to ask",
        format=lambda x: x.strip(),
    )
    answer: str = dspy.OutputField(
        prefix="Answer:",
        desc="answer to the question",
    )


## Loading Datasets

In [6]:
from random import sample
from dspy.datasets import DataLoader

dl = DataLoader()

In [7]:
tool_qa = dl.from_huggingface(
    f'{HF_USR_NAME}/' + hf_dataset_name,
    split="train",
    input_keys=("question", "answer"),
)

In [8]:
len(tool_qa)

100

In [9]:
import random
# set seed
random.seed(42)

train_idx = random.sample(range(len(tool_qa)), 40)
remaining_idx = list(set(range(len(tool_qa))) - set(train_idx))
test_idx = random.sample(remaining_idx, 60)

toolqa_train = [
    dspy.Example(question=example.question, answer=example.answer).with_inputs("question", "paper_id")
    for example in [tool_qa[i] for i in train_idx]
]
toolqa_test = [
    dspy.Example(question=example.question, answer=example.answer).with_inputs("question", "paper_id")
    for example in [tool_qa[i] for i in test_idx]
]

## Setting Up Tools

We'll setup `Avatar` modules for both signatures and all the `tools` can be used by each of the dataset. `Tool` is a pydantic model that Avatar expects the `tools` to be composed as more specifically it have 4 fields:

* `name` : Name of the tool
* `input_type` : Type of input the tool accepts
* `output_type` : Type of output the tool returns
* `tool` : The actual tool object

In [10]:
import os
import time
import uuid
import numpy as np
import jsonlines
from concurrent.futures import ProcessPoolExecutor
import sentence_transformers
import chromadb
import os.path as osp
from chromadb.config import Settings

EMBED_MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
CHROMA_PERSIST_DIRECTORY = osp.join(TOOL_QA_ROOT, "data/chroma_db/agenda")
CHROMA_COLLECTION_NAME = "all"
CHROMA_SERVER_HOST = "localhost"
CHROMA_SERVER_HTTP_PORT = "8000"
FILE_PATH = osp.join(TOOL_QA_ROOT, "data/external_corpus/agenda/agenda_descriptions_merged.jsonl")

def sentence_embedding(model, texts):
    embeddings = model.encode(texts)
    return embeddings

def create_chroma_db(chroma_server_host, chroma_server_http_port, collection_name):
    chroma_client = chromadb.Client(Settings(
        chroma_api_impl="rest",
        chroma_server_host=chroma_server_host,
        chroma_server_http_port=chroma_server_http_port,
    ))
    collection = chroma_client.get_or_create_collection(name=collection_name)
    return collection

def create_chroma_db_local(persist_directory, collection_name):
    
    chroma_client = chromadb.PersistentClient(path=persist_directory)

    collection = chroma_client.get_or_create_collection(name=collection_name)
    return collection

def insert_to_db(texts, model_name, cuda_idx, db):
    model = sentence_transformers.SentenceTransformer(model_name, device=f"cuda:{cuda_idx}")

    batch_embeddings = []
    batch_texts = []
    start_time = time.time()
    print(f"Total Articles to process: {len(texts)}, Current Thread: {cuda_idx}.")
    for i, text in enumerate(texts):
        # 2. generate embedding
        embeddings = sentence_embedding(model, text).tolist()

        batch_embeddings.append(embeddings)
        batch_texts.append(text)
        # 3. add to vectorstore per 500 articles or last article
        if i % 100 == 0 or i == len(texts)-1:
            batch_ids = [str(uuid.uuid1()) for _ in batch_texts]
            db.add(
                embeddings=batch_embeddings,
                documents=batch_texts,
                ids = batch_ids
            )
            batch_embeddings = []
            batch_texts = []
            print(f"Completed Processing article count: {i}, Current Thread: {cuda_idx}, Time took: {time.time() - start_time}.")
    print(f"Thread {cuda_idx} Completed. Total time took for thread: {time.time() - start_time}.")


# Multi-processing
def query_llm(query, is_local=True, start=None, end=None):
    EMBED_MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
    CHROMA_PERSIST_DIRECTORY = osp.join(TOOL_QA_ROOT, "data/chroma_db/agenda")
    os.makedirs(CHROMA_PERSIST_DIRECTORY, exist_ok=True)
    CHROMA_COLLECTION_NAME = "all"
    CHROMA_SERVER_HOST = "localhost"
    CHROMA_SERVER_HTTP_PORT = "8000"
    FILE_PATH = osp.join(TOOL_QA_ROOT, "data/external_corpus/agenda/agenda_descriptions_merged.jsonl")

    cuda_idxes = [0]
    number_of_processes = len(cuda_idxes)
    input_texts = []
    db = create_chroma_db_local(CHROMA_PERSIST_DIRECTORY, CHROMA_COLLECTION_NAME)
    with open(FILE_PATH, 'r') as f:
        for item in jsonlines.Reader(f):
            input_texts.append(item["event"])
    # input_texts = np.array_split(input_texts, number_of_processes)

    args = ((input_texts[i], EMBED_MODEL_NAME, cuda_idxes[i], is_local) for i in range(number_of_processes))

    # if there is no file under the directory "/localscratch/yzhuang43/ra-llm/retrieval_benchmark/data/chroma_db/agenda", insert the data into the db
    if len(os.listdir(CHROMA_PERSIST_DIRECTORY)) == 0:
        insert_to_db(input_texts, model_name=EMBED_MODEL_NAME, cuda_idx=0, db=db)

    input_paths = np.array_split(input_texts, number_of_processes)
    with ProcessPoolExecutor(number_of_processes) as executor:
        executor.map(insert_to_db, args)
    model = sentence_transformers.SentenceTransformer(EMBED_MODEL_NAME, device=f"cuda:0")
    query_embedding = sentence_embedding(model, query).tolist()
    results = db.query(query_embeddings=query_embedding, n_results=5)
    retrieval_content = [result for result in results['documents'][0]]
    # print(retrieval_content)
    retrieval_content = '\n\n'.join(retrieval_content)
    print('[query_llm]', query)
    return retrieval_content

query = "What is the Jessica's genda on March 7th, 2023?"
print(query_llm(query))

[query_llm] What is the Jessica's genda on March 7th, 2023?
Jessica has a job interview scheduled for July 26th, 2022 at 1 PM. The interview will take place at the Corporate Office Building, and is scheduled to end at 2 PM.

Jessica has a business meeting on January 22nd, 2022 at The Pacific Design Center. The meeting is scheduled to start at 3:00 PM and end at 4:00 PM.

Jessica will be attending a Late Night Movie on January 13, 2022, at a movie theater. The movie will start at 11:00 PM and end at 1:00 AM. She is looking forward to enjoying some popcorn and her favorite movie as she winds down her day.

Jessica is participating in a charity walk happening on April 22nd, 2022. The event is scheduled to start at 9:00 AM and will end at 11:00 AM. The walk will take place in Lincoln Park, and it's an opportunity for participants to raise money for a charitable organization while enjoying a morning walk. Jessica is sure to have a great time giving back to her community and supporting a goo

In [11]:
from dspy.predict.avatar import Tool, Avatar
from langchain_community.utilities import GoogleSerperAPIWrapper, ArxivAPIWrapper, WikipediaAPIWrapper
from langchain.tools import BaseTool, StructuredTool, tool

def RETRIEVE(query: str) -> str:
    """If you want to search for some personal agenda information, you can use this tool and input a natural language query. For example, RETRIEVE('What is the Jessica's genda on March 7th, 2023?') returns 'Jessica has a job interview scheduled for July 26th, 2022 at 1 PM...'."""
    return query_llm(query)

tools = [
    Tool(
        tool=StructuredTool.from_function(RETRIEVE),
        name="RETRIEVE",
        desc="If you want to search for some personal agenda information, you can use this tool and input a natural language query. For example, RETRIEVE('What is the Jessica's genda on March 7th, 2023?') returns 'Jessica has a job interview scheduled for July 26th, 2022 at 1 PM...'."
    )
]

Once we have defined our `tools`, we can now create an `Avatar` object by passing the `tools` and `signature`. It takes 2 more optional parameters `verbose` and `max_iters`. `verbose` is used to display the logs and `max_iters` is used to control the number of iterations in multi step execution. 

An avatar agent stops the tool usage iteration once it reaches `max_iters` or when it prompts `Finish`. You can also create custom tools too, all you need to make sure is:

* You pass is a class object.
* Implements `__init__` and `run` method.
* Must take 1 string a input and returns 1 string as output.

If your tool doesn't return or takes input a string then you can make a custom wrapper to take care of that for now. In future we'll try to enable a diverse tool usage.

In [12]:
actor_agent = Avatar(
    tools=tools,
    signature=ToolQASignature,
    verbose=False,
    max_iters=20
)

In [13]:
import time
import logging
from dataclasses import dataclass, field
from typing import Dict, List, Optional
from datetime import datetime
import tiktoken
from concurrent.futures import ThreadPoolExecutor, as_completed
import warnings
import copy
import tqdm
import logging
import warnings
import os

# Set up logging
# logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


# Disable all INFO logging
logging.getLogger().setLevel(logging.WARNING)

# Silence all loggers that might be chatty
loggers_to_silence = [
    "httpx",
    "httpcore",
    "openai",
    "arxiv",
    "dspy",
    "langchain",
    "langchain_community",
    "requests",
    "urllib3",
    "tiktoken",
    "asyncio",
    "faiss",
    "anthropic"
]

for logger_name in loggers_to_silence:
    logging.getLogger(logger_name).setLevel(logging.WARNING)

# Suppress specific warnings
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

os.environ['TOKENIZERS_PARALLELISM'] = 'false'  # Disable tokenizer parallelism warning

## Evaluation

Open enden QA tasks are hard to evaluate on rigid metrics like exact match. So, we'll be using an improvised LLM as Judge for the evaluation of our model on test set.

In [14]:
class Evaluator(dspy.Signature):
    """Please act as an impartial judge to evaluate whether the answer is correct based on the ground truth answer"""
    
    question: str = dspy.InputField(
        prefix="Question:",
        desc="question to ask",
    )
    reference_answer: str = dspy.InputField(
        prefix="Ground Truth Answer:",
        desc="Ground truth answer to the question.",
    )
    answer: str = dspy.InputField(
        prefix="Answer:",
        desc="Answer to the question given by the model.",
    )
    rationale: str = dspy.OutputField(
        prefix="Rationale:",
        desc="Explanation of why the answer is correct or incorrect.",
    )
    is_correct: float = dspy.OutputField(
        prefix="Correct:",
        desc="Whether the answer is correct. Give 0 if incorrect, 1 if correct, (0, 1) if partially correct.",
    )


evaluator = dspy.TypedPredictor(Evaluator)


def metric(example, prediction, trace=None):
    acc = float(
        evaluator(
            question=example.question,
            answer=prediction.answer,
            reference_answer=example.answer
        ).is_correct
    ) 
    print(prediction.answer, '|', example.answer, '=>', acc)
    return acc

In [15]:
print(toolqa_train[0])
metric(toolqa_train[0], prediction=dspy.Example(answer='73 days'))

Example({'question': 'How many dates in the agenda table have Alexander scheduled?', 'answer': '73'}) (input_keys={'question', 'paper_id'})
73 days | 73 => 1.0


1.0

For evaluation we can't use `dspy.Evaluate`, reason being that `Avatar` changes it's signature per iteration by adding the actions and it's results to it as fields. So we can create our own hacky thread safe evaluator for it.

In [16]:
import time
import logging
from dataclasses import dataclass, field
from typing import Dict, List, Optional
from datetime import datetime
import tiktoken
from concurrent.futures import ThreadPoolExecutor, as_completed
import warnings
import copy

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

@dataclass
class APICallMetrics:
    timestamp: datetime
    tool_name: str
    tokens_in: int = 0
    tokens_out: int = 0
    execution_time: float = 0.0

@dataclass
class AvatarMetrics:
    total_calls: int = 0
    total_tokens_in: int = 0
    total_tokens_out: int = 0
    total_execution_time: float = 0.0
    calls_by_tool: Dict[str, int] = field(default_factory=dict)
    api_call_history: List[APICallMetrics] = field(default_factory=list)
    
    def add_call(self, metrics: APICallMetrics):
        self.total_calls += 1
        self.total_tokens_in += metrics.tokens_in
        self.total_tokens_out += metrics.tokens_out
        self.total_execution_time += metrics.execution_time
        self.calls_by_tool[metrics.tool_name] = self.calls_by_tool.get(metrics.tool_name, 0) + 1
        self.api_call_history.append(metrics)
    
    def merge(self, other: 'AvatarMetrics'):
        """Merge another AvatarMetrics instance into this one"""
        self.total_calls += other.total_calls
        self.total_tokens_in += other.total_tokens_in
        self.total_tokens_out += other.total_tokens_out
        self.total_execution_time += other.total_execution_time
        for tool, count in other.calls_by_tool.items():
            self.calls_by_tool[tool] = self.calls_by_tool.get(tool, 0) + count
        self.api_call_history.extend(other.api_call_history)

    def estimate_cost(self, model_name: str = "gpt-4") -> float:
        pricing = {
            "gpt-4": {"input": 2.5, "output": 10.0},
        }
        if model_name not in pricing:
            raise ValueError(f"Unknown model: {model_name}")
        
        rates = pricing[model_name]
        input_cost = (self.total_tokens_in / 1000000) * rates["input"]
        output_cost = (self.total_tokens_out / 1000000) * rates["output"]
        return input_cost + output_cost

class AvatarWithMetrics(Avatar):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.metrics = AvatarMetrics()
        self.tokenizer = tiktoken.encoding_for_model("gpt-4")
    
    def _count_tokens(self, text: str) -> int:
        try:
            return len(self.tokenizer.encode(str(text)))
        except Exception as e:
            logger.warning(f"Error counting tokens: {e}")
            return 0

    def _wrapped_tool_call(self, tool, input_text: str) -> str:
        start_time = time.time()
        tokens_in = self._count_tokens(input_text)
        
        try:
            result = tool.run(input_text)
        except Exception as e:
            logger.error(f"Tool execution error: {e}")
            raise
        finally:
            execution_time = time.time() - start_time
            tokens_out = self._count_tokens(str(result))
            
            metrics = APICallMetrics(
                timestamp=datetime.now(),
                tool_name=tool.name,
                tokens_in=tokens_in,
                tokens_out=tokens_out,
                execution_time=execution_time
            )
            self.metrics.add_call(metrics)
            
        return result

    def __call__(self, *args, **kwargs):
        start_time = time.time()
        result = super().__call__(*args, **kwargs)
        total_time = time.time() - start_time
        
        metrics = APICallMetrics(
            timestamp=datetime.now(),
            tool_name="main_llm",
            tokens_in=self._count_tokens(str(args) + str(kwargs)),
            tokens_out=self._count_tokens(str(result)),
            execution_time=total_time
        )
        self.metrics.add_call(metrics)
        
        return result

def multi_thread_executor(test_set, signature, num_threads=60):
    total_score = 0
    total_examples = len(test_set)
    combined_metrics = AvatarMetrics()

    start_time = time.time()
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = []
        for example in test_set:
            def process_with_metrics(example=example):
                try:
                    avatar = AvatarWithMetrics(signature, tools=tools, verbose=False, max_iters=10)
                    prediction = avatar(**example.inputs().toDict())
                    return metric(example, prediction), avatar.metrics
                except Exception as e:
                    print(e)
                    return 0, AvatarMetrics()

            futures.append(executor.submit(process_with_metrics))

        for future in tqdm.tqdm(futures, total=total_examples, desc="Processing examples"):
            score, metrics = future.result()
            total_score += score
            # Only combine token counts and call counts, not execution times
            combined_metrics.total_calls += metrics.total_calls
            combined_metrics.total_tokens_in += metrics.total_tokens_in
            combined_metrics.total_tokens_out += metrics.total_tokens_out
            for tool, count in metrics.calls_by_tool.items():
                combined_metrics.calls_by_tool[tool] = combined_metrics.calls_by_tool.get(tool, 0) + count
            combined_metrics.api_call_history.extend(metrics.api_call_history)
    
    total_execution_time = time.time() - start_time
    combined_metrics.total_execution_time = total_execution_time

    avg_metric = total_score / total_examples
    return avg_metric, combined_metrics

def single_thread_executor(test_set, signature):
    total_score = 0
    total_examples = len(test_set)
    combined_metrics = AvatarMetrics()

    for example in tqdm.tqdm(test_set, desc="Processing examples"):
        try:
            avatar = AvatarWithMetrics(signature, tools=tools, verbose=False, max_iters=10)
            prediction = avatar(**example.inputs().toDict())
            score = metric(example, prediction)
            total_score += score
            # Combine metrics from this run
            for call in avatar.metrics.api_call_history:
                combined_metrics.add_call(call)
        except Exception as e:
            print(e)

    avg_metric = total_score / total_examples
    return avg_metric, combined_metrics

def format_metrics_report(metrics: AvatarMetrics, model_name: str = "gpt-4") -> str:
    cost = metrics.estimate_cost(model_name)
    
    report = f"""
Avatar Execution Metrics Report
==============================
Execution Time: {metrics.total_execution_time:.2f} seconds
Total API Calls: {metrics.total_calls}
Total Tokens: {metrics.total_tokens_in + metrics.total_tokens_out:,} ({metrics.total_tokens_in:,} in, {metrics.total_tokens_out:,} out)
Estimated Cost: ${cost:.4f}

Average Time per Call: {metrics.total_execution_time / metrics.total_calls:.2f} seconds

Tool Usage Breakdown:
-------------------
"""
    for tool, count in sorted(metrics.calls_by_tool.items()):
        report += f"{tool}: {count} calls\n"

    return report

In [17]:
score, metrics = multi_thread_executor(toolqa_test, ToolQASignature)

Processing examples:   0%|          | 0/60 [00:00<?, ?it/s]

[query_llm] How many dates in the agenda table have Harper scheduled?
[query_llm] What events does Andrew have on 2022/06/12 in the agenda table?
5 | 94 => 0.0
Andrew has no events on 2022/06/12 in the agenda table. | Artisanal candy making class => 0.0
[query_llm] Check Hector's availability on 2022/01/12 from 9:00 AM to 6:00 PM in the agenda table.
Hector is available for a meeting on 2022/01/12 from 9:00 AM to 6:00 PM as there are no scheduled events for that day in the agenda table. | 9:00 AM-9:00 PM, 11:00 PM-6:00 PM => 1.0
[query_llm] How many dates in the agenda table have Imogen scheduled?
5 | 88 => 0.0
[query_llm] Number of people between 3:00 PM and 4:30 PM on 2022/06/21 in the agenda table
There is no information available for the specified date and time in the agenda table. | 10 => 0.0
[query_llm] Count the number of people scheduled between 9:00 AM and 2:00 PM on 2022/06/07 in the agenda table.
[query_llm] Number of people between 3:00 PM and 4:00 PM on 2022/05/27 in the a

Processing examples:   2%|▏         | 1/60 [01:11<1:09:57, 71.14s/it]

[query_llm] Count the number of events on 2022/01/06 in the agenda table.
0 | 30 => 0.0
[query_llm] How many people are between 12:00 PM and 3:00 PM on 2022/03/13 in the agenda table?
There is no information available for events between 12:00 PM and 3:00 PM on 2022/03/13 in the agenda table. | 6 => 0.0
[query_llm] How many dates in the agenda table have Elsie scheduled?
4 | 83 => 0.0
[query_llm] What events does Benjamin have on 2022/06/10 in the agenda table?
Benjamin does not have any events scheduled on 2022/06/10 in the agenda table. | Join a fitness bootcamp => 0.0
[query_llm] Check Elizabeth's availability on 2022/01/10 from 9:00 AM to 6:00 PM in the agenda table.
Elizabeth is available for a meeting on 2022/01/10 from 9:00 AM to 6:00 PM as there are no conflicting appointments in the agenda table. | 9:00 AM-10:00 AM, 4:00 PM-8:00 PM, 10:00 PM-6:00 PM => 0.0
[query_llm] Check availability for Joshua on 2022/12/02 from 9:00 AM to 6:00 PM in the agenda table.
Joshua is available fo

Processing examples: 100%|██████████| 60/60 [01:31<00:00,  1.53s/it] 

[query_llm] Count the number of people scheduled between 6:00 AM and 12:00 PM on 2022/08/28 in the agenda table.
[query_llm] Check Victoria's availability on 2022/08/05 from 9:00 AM to 6:00 PM in the agenda table.
Alice is available for a meeting on 2022/01/06 from 9:00 AM to 6:00 PM as there are no events scheduled for her on that day. | 9:00 AM-1:00 PM, 3:00 PM-6:00 PM => 0.0
Victoria is available for a meeting on 2022/08/05 from 9:00 AM to 2:00 PM and from 4:00 PM to 6:00 PM. | 9:00 AM-2:00 PM, 4:00 PM-6:00 PM => 1.0
There are no people scheduled between 6:00 AM and 12:00 PM on 2022/08/28 in the agenda table. | 7 => 0.0


In [18]:
# print(f"Average Score on ArxivQA before opitmization: {aqa_score:.2f}")
print(f"Test Score: {score:.2f}")
print(format_metrics_report(metrics))

Test Score: 0.05

Avatar Execution Metrics Report
Execution Time: 92.71 seconds
Total API Calls: 60
Total Tokens: 25,694 (1,745 in, 23,949 out)
Estimated Cost: $0.2439

Average Time per Call: 1.55 seconds

Tool Usage Breakdown:
-------------------
main_llm: 60 calls



## Optimization

For the optimization of the `Actor` we'll be using `AvatarOptimizer`. It's a DSPy implementation of the [Avatar](https://github.com/zou-group/avatar/) method that optimizes the `Actor` for the given `tools` using a comparator module that optimizes Actor instruction. Note, that Actor is the Module that directs tool execution and flow, it's not the signature that we are passing. It doesn't optimize the instruction of the signature we pass. It takes the following parameters:

* `metric`: Metric that we'll be optimizing for
* `max_iters`: Maximum number of iterations for the optimizer
* `lower_bound`: Lower bound for the metric to classify example as negative
* `upper_bound`: Upper bound for the metric to classify example as positive
* `max_positive_inputs`: Maximum number of positive inputs sampled for comparator
* `max_negative_inputs`: Maximum number of negative inputs sampled for comparator
* `optimize_for`: Whether we want to maximize the metric or minimize it during optimization

Once the optimizer is done we can get the optimized actor and use it for the evaluation.

In [19]:
from new_optimizer import AvatarOptimizerWithMetrics

iterative_monkey = AvatarOptimizerWithMetrics(
    metric=metric,
    max_iters=2,
    max_negative_inputs=20,
    max_positive_inputs=20,
    upper_bound=0,
    lower_bound=0 # special case when we need some perturbation if no pos/neg
)

In [20]:
result = iterative_monkey.compile(
    student=actor_agent,
    trainset=toolqa_train
)

Processing examples:   0%|          | 0/40 [00:00<?, ?it/s]

[query_llm] How many dates in the agenda table have Ella scheduled?
4 | 85 => 0.0
[query_llm] What events does Emily have on 2022/09/19 in the agenda table?
Emily has no events scheduled on 2022/09/19 in the agenda table. | Theatre performance, Art Walk => 0.0
[query_llm] What events does Jasmine have on 2022/05/10 in the agenda table?
Jasmine does not have any events scheduled on 2022/05/10 in the agenda table. | Vegan Food Festival => 0.0
[query_llm] What events does Jade have on 2022/01/25 in the agenda table?
Jade has no events on 2022/01/25 in the agenda table. | Kids concert => 0.0
[query_llm] Count the number of events on 2022/10/11 in the agenda table.
There are no events listed on 2022/10/11 in the agenda table. | 23 => 0.0
[query_llm] Count the number of events on 2022/03/09 in the agenda table.
0 | 24 => 0.0
[query_llm] How many dates in the agenda table have Hector scheduled?
[query_llm] Count the number of events on 2022/12/01 in the agenda table.
5 | 92 => 0.0
[query_llm]

Processing examples:   2%|▎         | 1/40 [00:48<31:49, 48.96s/it]

4 | 73 => 0.0
3 | 88 => 0.0
[query_llm] How many dates in the agenda table have Faith scheduled?
5 | 85 => 0.0


Processing examples:  12%|█▎        | 5/40 [00:49<04:18,  7.38s/it]

...

Processing examples:  10%|█         | 4/40 [00:49<06:51, 11.42s/it]

5 | 74 => 0.0
4 | 86 => 0.0
There are no events listed on 2022/11/12 in the agenda table. | 24 => 0.0
5 | 76 => 0.0
There are no events listed on 2022/09/25 in the agenda table. | 24 => 0.0
Emma is available for a meeting on 2022/06/14 between 10:00 AM and 6:00 PM. | 9:00 AM-6:30 PM, 7:30 PM-6:00 PM => 1.0
Millie has no events scheduled on 2022/12/07 in the agenda table. | Theatre play => 0.0
Jasmine does not have any events scheduled on 2022/05/10 in the agenda table. | Vegan Food Festival => 0.0
Sophie is scheduled on 3 dates in the agenda table. | 83 => 0.0
There are no events listed on 2022/05/06 in the agenda table. | 21 => 0.0
[query_llm] List all events scheduled between 12:00 PM and 2:00 PM on 2022/09/13 in the agenda table.
No relevant data found for the specified date and time range in the agenda table. | 6 => 0.0
0 | 3 => 0.0
There are no people scheduled between 8:00 PM and 10:00 PM on 2022/06/06 in the agenda table. | 7 => 0.0
There are no events scheduled between 8:00 PM 

Processing examples: 100%|██████████| 40/40 [00:58<00:00,  1.47s/it]

There are no people scheduled between 12:00 PM and 2:00 PM on 2022/09/13 in the agenda table. | 11 => 0.0
Average Score: 0.05


Generated new instruction: New Instruction: You will be given `Tools`, which will be a list of tools to use to accomplish the `Goal`. Your task is to decide which tool to use and what input values to provide based on the user query. To enhance performance, focus on crafting specific and well-aligned input queries that match the tool's capabilities. Ensure that each query is directly related to the agenda table to facilitate accurate information retrieval. You may use a tool multiple times with different input queries if applicable, or opt to provide the final answer directly without using any tools.

To address challenges with negative inputs, implement an enhanced query parsing mechanism. This mechanism should be capable of understanding and interpreting the user's intent, even if the query is not perfectly structured. Introduce a contextual understanding layer that can infer the user's needs from partial or ambiguous queries, thereby improving the relevance of the tool outputs. Addit

Now we can evaluate our actor module, for this we've provided an implementation of thread safe evaluator that we above as part of class method of `AvatarOptimizer`.

In [ ]:
optimized_actor_agent = result["agent"]
optimization_metrics = result["metrics"]

# Now you can process the metrics
print(f"Total optimization cost: ${optimization_metrics['total_cost']:.4f}")
print(f"Final score achieved: {optimization_metrics['final_score']:.3f}")

# Analyze per-iteration performance
for iteration in optimization_metrics['iteration_details']:
    print(f"\nIteration {iteration['iteration']}:")
    print(f"Score: {iteration['score']:.3f}")
    print(f"Comparator tokens in: {iteration['comparator_metrics']['tokens_in']}")
    print(f"Comparator tokens out: {iteration['comparator_metrics']['tokens_out']}")
    print(f"Feedback tokens in: {iteration['feedback_metrics']['tokens_in']}")
    print(f"Feedback tokens out: {iteration['feedback_metrics']['tokens_out']}")
    print(f"Execution time: {iteration['total_iteration_time']:.2f}s")

Total optimization cost: $1.0385
Final score achieved: 0.050

Iteration 0:
Score: 0.025
Comparator tokens in: 14544
Comparator tokens out: 373
Feedback tokens in: 509
Feedback tokens out: 281
Execution time: 63.11s

Iteration 1:
Score: 0.050
Comparator tokens in: 16200
Comparator tokens out: 382
Feedback tokens in: 699
Feedback tokens out: 297
Execution time: 80.10s


In [22]:
batch_num = 4
iterative_monkey.thread_safe_evaluator_batch(toolqa_test, optimized_actor_agent,batch_num)

Processing batch 1 of 4...


Processing examples:   0%|          | 0/60 [00:00<?, ?it/s]

[query_llm] Count the number of events on 2022/01/06 in the agenda table.
[query_llm] Events for Brian on 2022/03/17 in the agenda table
[query_llm] Count the number of dates Harper is scheduled in the agenda table.
[query_llm] Count the number of dates Charles has scheduled in the agenda table.
[query_llm] Find available time slots for Alice on 2022/01/06 between 9:00 AM and 6:00 PM in the agenda table.
[query_llm] Events for Andrew on 2022/06/12 in the agenda table
[query_llm] Count the number of dates in the agenda table where Hannah has scheduled events.
[query_llm] Check Faith's availability on 2022/04/01 between 9:00 AM and 6:00 PM in the agenda table.
[query_llm] Count the number of events on 2022/12/18 in the agenda table.
[query_llm] Count the number of people scheduled between 9:00 AM and 2:00 PM on 2022/06/07 in the agenda table.
[query_llm] Count the number of events on 2022/01/08 in the agenda table.
[query_llm] Count the number of dates Christopher has scheduled in the ag

Processing examples:   2%|▏         | 1/60 [00:35<35:04, 35.68s/it]

There are no events listed on 2022/01/06 in the agenda table. | 30 => 0.0
"There are no events listed on 2022/01/08 in the agenda table." | 26 => 0.0
[query_llm] Sophia's events on 2022/03/10 in the agenda table
[query_llm] Count the number of people scheduled between 3:00 PM and 4:00 PM on 2022/05/27 in the agenda table.


...

Processing examples:  33%|███▎      | 20/60 [01:15<00:54,  1.35s/it]

"There are no events listed on 2022/01/08 in the agenda table." | 26 => 0.0
[query_llm] Count the number of dates Harper is scheduled in the agenda table.
5 | 94 => 0.0
[query_llm] Check Hector's availability on 2022/01/01 between 9:00 AM and 6:00 PM in the agenda table.
Hector is available all day on 2022/01/01 as there are no scheduled events for that date in the agenda table. | 9:00 AM-6:30 PM, 8:30 PM-6:00 PM => 0.0
[query_llm] Count the number of people scheduled between 9:00 AM and 3:00 PM on 2022/06/20 in the agenda table.
[query_llm] Count the number of people scheduled between 3:00 PM and 4:30 PM on 2022/06/21 in the agenda table.
[query_llm] Events for James on 2022/10/23
There are no entries in the agenda table for the specified date and time range. | 10 => 0.0
James has no events on 2022/10/23 in the agenda table. | Mindfulness meditation class => 0.0
[query_llm] Events for Phoebe on 2022/12/19 in the agenda table
Phoebe has no events on 2022/12/19 in the agenda table. | Ch

Processing examples: 100%|██████████| 60/60 [01:17<00:00,  1.30s/it]

[query_llm] List all events scheduled between 9:00 AM and 3:00 PM on 2022/06/20 in the agenda table.
0 | 14 => 0.0


0.058333333333333334